## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv  # Load environment variables from a .env file (e.g., API keys, secrets)
from agents import Agent, Runner, trace, function_tool  # Components for agent-based workflows, tracing, and custom tools
from openai.types.responses import ResponseTextDeltaEvent  # Handles text streaming/delta events from OpenAI API responses
from typing import Dict  # Provides type hinting for dictionaries
import sendgrid  # Send emails using SendGrid's API
import os  # Access OS-level functionality, especially environment variables
from sendgrid.helpers.mail import Mail, Email, To, Content  # Helper classes from SendGrid to construct emails (sender, recipient, subject, content)
import asyncio  # Enables asynchronous programming (useful for concurrent tasks or non-blocking requests)

In [2]:
# Load environment variables from a .env file, allowing for configuration without hardcoding
load_dotenv(override = True)

True

In [3]:
def send_test_email():
    """
    Sends a test email using the SendGrid API.

    <b>*Parameters*</b>
    - None: The function does not take any input parameters.
    
    <b>*Returns*</b>
    - None explicitly. 
    - Prints the email sending status code to the console.
      - 202 = Email successfully queued for sending
      - Other codes = Indicates an error (e.g., authentication failure, invalid email)

    <b>*Logic*</b>
    1. Retrieve the SendGrid API key from environment variables for secure authentication.
    2. Define the sender’s email address (must be verified in SendGrid).
    3. Define the recipient’s email address (target where the email will be sent).
    4. Define the email content (plain text in this case).
    5. Construct the Mail object with sender, receiver, subject, and body.
    6. Convert the Mail object into a JSON-compatible format using `.get()`.
    7. Send the email request through SendGrid’s API.
    8. Print the response status code to check if the email was sent successfully.
    """
    
    # Create a SendGrid client object using your API key.
    # The API key here is read from an environment variable called "SENDGRID_API_KEY".
    # This keeps your private key safe instead of typing it directly into the code.
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get("SENDGRID_API_KEY"))
    
    # Define the sender email address (MUST be verified in your SendGrid account).
    from_email = Email(email = "siddharthwolverine@gmail.com")
    
    # Define the recipient email address (the person who should receive the email).
    to_email = To(email = "siddharth13101999singh@gmail.com")
    
    # Define the actual message content.
    # - mime_type = "text/plain" means this email will only contain plain text.
    # - content = "This is an important test email" is the body of the email.
    content = Content(mime_type = "text/plain", content = "This is an important test email")
    
    # Create the email message itself.
    # - from_email: who is sending the email
    # - to_email: who will receive the email
    # - "Test email": the subject of the email
    # - content: what is written inside the email body
    # ".get()" converts the object into the proper format (JSON) so that SendGrid can understand it.
    mail = Mail(from_email = from_email, to_emails = to_email, subject = "Test email", plain_text_content = content).get()
    
    # Actually send the email by making a POST request to SendGrid's mail API.
    # 'request_body = mail' means the email details created above are sent as part of the request.
    response = sg.client.mail.send.post(request_body = mail)
    
    # Print the status code returned by the server.
    # - 202 means the email was successfully accepted by SendGrid.
    # - Any other status codes indicate something went wrong.
    print(response.status_code)

# Call the function to run it.
send_test_email()

202


### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

## Step 1: Agent workflow

In [4]:
# Instruction set for a professional tone:
# The AI should act like a serious sales agent, writing formal, professional cold emails.
instructions1 = "You are a sales agent working for CompanionAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

# Instruction set for a humorous tone:
# The AI should act like a witty sales agent, writing engaging cold emails with humor to spark responses.
instructions2 = "You are a humorous, engaging sales agent working for CompanionAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

# Instruction set for a concise tone:
# The AI should act like a busy sales agent, writing short and direct cold emails that respect the reader’s time.
instructions3 = "You are a busy sales agent working for CompanionAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# Create an instance of Agent named "Professional Sales Agent". This agent will follow the instructions defined in instructions1,
# which asks for a professional and serious tone in cold emails. The model specified for generating responses is "gpt-4o-mini".
sales_agent1 = Agent(name = "Professional Sales Agent", instructions = instructions1, model = "gpt-4o-mini")

# Create an instance of Agent named "Engaging Sales Agent". This agent will follow the instructions defined in instructions2,
# which asks for a humorous, witty, and engaging email style. The model specified for generating responses is "gpt-4o-mini".
sales_agent2 = Agent(name = "Engaging Sales Agent", instructions = instructions2, model = "gpt-4o-mini")

# Create an instance of Agent named "Busy Sales Agent". This agent will follow the instructions defined in instructions3,
# which asks for concise, direct, and to-the-point cold emails. The model specified for generating responses is "gpt-4o-mini".
sales_agent3 = Agent(name = "Busy Sales Agent", instructions = instructions3, model = "gpt-4o-mini")

In [6]:
# Run the sales_agent1 agent in streamed mode using Runner. This starts the agent with the input prompt: "Write a cold sales email".
result = Runner.run_streamed(starting_agent = sales_agent1, input = "Write a cold sales email")

# Iterate asynchronously over streaming events returned by the agent. This loop waits for the agent's response events in real-time.
async for event in result.stream_events():
    # Check if the event is a raw response event and its data is of type ResponseTextDeltaEvent,
    # which represents partial incremental chunks of the response text.
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        # Print the delta (small piece of generated text) immediately, with no newline and flushing the output 
        # buffer to show streaming results live.
        print(event.data.delta, end = "", flush = True)

Subject: Streamline Your SOC 2 Compliance Process with CompanionAI

Hi [Recipient's Name],

I hope this message finds you well. 

As organizations face increasing scrutiny regarding data security and privacy, achieving SOC 2 compliance can be a complex and time-consuming process. At CompanionAI, we understand this challenge and have developed an innovative SaaS tool designed to simplify and automate your compliance journey.

Our AI-powered solution not only helps streamline the preparation for audits but also provides ongoing monitoring and guidance to ensure your organization maintains compliance at all times. By leveraging our tool, many of our clients have reduced compliance preparation time by up to 50%, allowing them to focus more on their core business.

I would love to schedule a brief call to discuss how CompanionAI can support your organization in achieving and maintaining SOC 2 compliance efficiently. Are you available for a quick chat this week?

Thank you for considering Co

In [7]:
# The message to be sent as input to all sales agents, requesting a cold sales email
message = "Write a cold sales email"

# Begin a traced context, labeled "Parallel cold emails"
# This helps track or debug the execution within this block
with trace("Parallel cold emails"):
    # Run all three sales agents concurrently in parallel using asyncio.gather
    # Each Runner.run call sends the message to a specific sales agent to generate a response
    # 'await' ensures the program pauses here until all three asynchronous tasks complete
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),  # Professional Sales Agent
        Runner.run(sales_agent2, message),  # Engaging Sales Agent
        Runner.run(sales_agent3, message),  # Busy Sales Agent
    )

# Collect the final output (the generated cold emails) from each of the returned result objects
outputs = [result.final_output for result in results]

# Print each of the generated emails, separated by two newline characters for readability
for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC 2 Compliance Process with CompanionAI

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name], reaching out from CompanionAI, where we specialize in empowering organizations like yours to simplify their SOC 2 compliance journey.

As you know, maintaining SOC 2 compliance can be a complex and time-consuming endeavor. Our AI-powered SaaS tool is designed to automate key processes, making it easier for your team to manage compliance requirements and prepare for audits efficiently. 

Here are a few ways CompanionAI can support your organization:

- **Automated Documentation**: Generate and maintain the necessary documentation effortlessly, reducing the burden on your team.
- **Real-time Monitoring**: Stay up-to-date with compliance status and receive actionable insights to address potential issues before they arise.
- **Audit Preparation**: Simplify audit processes with organized, easily accessible data, allowing for a smoother review.

I w

In [8]:
# Create an Agent instance named "sales_picker"
# This agent’s role is to choose the best cold sales email from multiple options.
# The instructions tell the agent to imagine being a customer and pick the email most likely to get a response.
# The agent should only reply with the chosen email and not provide any explanation.
# The language model used for this agent is "gpt-4o-mini".
sales_picker = Agent(name = "sales_picker",
    instructions = (
        "You pick the best cold sales email from the given options. "
        "Imagine you are a customer and pick the one you are most likely to respond to. "
        "Do not give an explanation; reply with the selected email only."
    ),
    model = "gpt-4o-mini"
)

In [9]:
# Input prompt to all sales agents asking them to write a cold sales email
message = "Write a cold sales email"


# Begin a traced context labeled "Selection from sales people" for debugging or profiling purposes
with trace("Selection from sales people"):
    # Run all three sales agents concurrently to generate their versions of cold sales emails
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),  # Professional Sales Agent
        Runner.run(sales_agent2, message),  # Engaging Sales Agent
        Runner.run(sales_agent3, message),  # Busy Sales Agent
    )
    
    # Extract the generated email text from each agent's result
    outputs = [result.final_output for result in results]

    # Prepare a combined string of all cold sales emails separated by "\n\nEmail:\n\n"
    # This formats the emails so that the sales_picker agent can evaluate them clearly
    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    # Use the sales_picker agent to select the best email from the compiled list
    best = await Runner.run(sales_picker, emails)

    # Print the best sales email chosen by the sales_picker agent
    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Let's Make SOC2 Compliance Less Painful (and More Fun)!

Hey [Recipient's Name],

I hope this email finds you well and enjoying a coffee that hasn’t gone cold! ☕️

Let’s talk SOC2 compliance—because who doesn’t love a good audit in their life? (Just kidding!) Let’s be real; it can feel like wading through molasses. But what if I told you there’s a way to speed things up without losing your mind (or your hair)?

At CompanionAI, we like to think of ourselves as your compliance sidekick—minus the spandex. With our AI-powered tool, we can help you achieve and maintain SOC2 compliance faster than you can say “Where are all my documents?” 🥴

Imagine:
- Real-time monitoring that doesn’t require a crystal ball 🔮
- Automated reporting that makes spreadsheets jealous
- And a sprinkle of “Wow, we can actually do this without pulling all-nighters!”

Let’s chat! I promise it’ll be more fun than trying to figure out that one vendor-required document you’ve been avoiding.



Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [10]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for CompanionAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [11]:
@function_tool
def send_email(body : str):
    """
    Send out an email with the given body to all sales prospects.

    <b>*Parameters*</b>
    - body (str): The content of the email message to be sent.

    <b>*Returns*</b>
    - dict: A dictionary containing the status of the email sending operation.

    <b>*Logic*</b>
    1. Create a SendGrid client authenticated using the API key stored in environment variables.
    2. Define the sender email address (must be a verified sender in SendGrid).
    3. Define the recipient email address (where the email will be delivered).
    4. Create the email content using the provided body text and set MIME type to plain text.
    5. Construct the Mail object combining sender, recipient, subject, and content.
    6. Send the email via SendGrid's API using a POST request.
    7. Return a success status dictionary indicating the email was sent.
    """

    # Initialize the SendGrid client using the API key from the environment
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get("SENDGRID_API_KEY"))

    # Set the verified sender email address
    from_email = Email(email = "siddharthwolverine@gmail.com")  # Change this to your verified sender

    # Set the recipient email address
    to_email = To(email = "siddharth13101999singh@gmail.com")  # Change this to your recipient

    # Define the email content with the given text body and MIME type "text/plain"
    content = Content(mime_type = "text/plain", content = body)

    # Create the email message object with sender, recipient, subject, and content
    mail = Mail(from_email = from_email, to_emails = to_email, subject = "Test email", plain_text_content = content).get()

    # Send the email through SendGrid's mail API
    sg.client.mail.send.post(request_body = mail)

    # Return a success response to indicate the email was sent
    return {"status" : "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [12]:
# Let's look at it
send_email

FunctionTool(name='send_email', description="Send out an email with the given body to all sales prospects.\n\n<b>*Parameters*</b>\n- body (str): The content of the email message to be sent.\n\n<b>*Returns*</b>\n- dict: A dictionary containing the status of the email sending operation.\n\n<b>*Logic*</b>\n1. Create a SendGrid client authenticated using the API key stored in environment variables.\n2. Define the sender email address (must be a verified sender in SendGrid).\n3. Define the recipient email address (where the email will be delivered).\n4. Create the email content using the provided body text and set MIME type to plain text.\n5. Construct the Mail object combining sender, recipient, subject, and content.\n6. Send the email via SendGrid's API using a POST request.\n7. Return a success status dictionary indicating the email was sent.", params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'objec

### And you can also convert an Agent into a tool

In [13]:
# Convert the sales_agent1 Agent instance into a callable tool
# This allows invoking sales_agent1 through a standardized tool interface
# The tool is named "sales_agent1" and has a description "Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = "Write a cold sales email")

tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134cbb4fa60>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [14]:
# Description used for all sales agent tools
description = "Write a cold sales email"

# Convert the three sales agent instances into callable tools
# Each tool has a unique name and shares the same description
tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = description)
tool2 = sales_agent2.as_tool(tool_name = "sales_agent2", tool_description = description)
tool3 = sales_agent3.as_tool(tool_name = "sales_agent3", tool_description = description)

# Create a list of all tools including the send_email function_tool
tools = [tool1, tool2, tool3, send_email]

# Display the list of tools to verify they are set up correctly
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134f01ced40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134cbb4ede0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

## And now it's time for our Sales Manager - our planning agent

In [15]:
# Instructions given to the Sales Manager agent
# This agent is responsible for orchestrating the process of finding and sending the best cold sales email
instructions = """
You are a Sales Manager at CompanionAI. Your goal is to find the single best cold sales email using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
3. Use the send_email tool to send the best email (and only the best email) to the user.

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

# Create the Sales Manager agent using the instructions above
# The agent is given access to all tools including sales agents and the send_email tool
# "gpt-4o-mini" is specified as the language model to generate and evaluate emails
sales_manager = Agent(name = "Sales Manager", instructions = instructions, tools = tools, model = "gpt-4o-mini")

# The message input sent to the Sales Manager agent to trigger the workflow
message = "Send a cold sales email addressed to 'Dear CEO'"

# Use a trace context for monitoring or debugging the Sales Manager's execution
with trace("Sales manager"):
    # Run the Sales Manager agent asynchronously with the given message
    # This will execute the whole workflow of generating, selecting, and sending the email
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [16]:
# Instructions defining the task for the Email Subject Writer agent. This agent's role is to write a subject line for a cold sales email
# It receives the main message content and needs to create a subject likely to get a response from the recipient.
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

# Instructions defining the task for the HTML Email Body Converter agent. This agent converts a plain text email body (which might contain markdown)
# into a clear, simple, and compelling HTML formatted email body.
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

# Creating an agent instance for writing email subjects 'name' gives the agent an identity 'instructions' defines what this agent should do
# 'model' specifies which AI model powers this agent
subject_writer = Agent(name = "Email subject writer", instructions = subject_instructions, model = "gpt-4o-mini")

# Convert the subject_writer agent into a reusable tool Tools can be called easily within the overall agentic AI workflow
# 'tool_name' is the identifier used to invoke this particular tool 'tool_description' briefly explains this tool's purpose
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description = "Write a subject for a cold sales email")

# Creating an agent instance for converting plain text email to HTML
html_converter = Agent(name = "HTML email body converter", instructions = html_instructions, model = "gpt-4o-mini")

# Convert the html_converter agent into a reusable tool
html_tool = html_converter.as_tool(tool_name = "html_converter", tool_description = "Convert a text email body to an HTML email body")

In [17]:
@function_tool
def send_html_email(subject : str, html_body : str) -> Dict[str, str]:
    """
    Sends an HTML-formatted email using the SendGrid API.

    <b>*Parameters*</b>
    - subject (str): The subject line of the email to be sent.
    - html_body (str): The HTML content that will be used as the body of the email.

    <b>*Returns*</b>
    - Dict[str, str]: A dictionary indicating the status of the operation (e.g., {"status": "success"}).

    <b>*Logic*</b>
    1. Initialize the SendGrid client using the required API key, retrieved securely from environment variables.
    2. Define and validate the sender's email address (must be verified in SendGrid).
    3. Define the recipient's email address.
    4. Construct the email content using the provided HTML body with MIME type "text/html".
    5. Assemble the Mail object with sender, recipient, subject, and HTML content.
    6. Send the email using SendGrid's Mail API with a POST request.
    7. Return a status message indicating the outcome ("success" if the operation was completed).
    """

    # Initialize the SendGrid client using the API key from the environment
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get("SENDGRID_API_KEY"))

    # Set the verified sender email address (must be verified in your SendGrid dashboard)
    from_email = Email(email = "siddharthwolverine@gmail.com")  # Change this to your verified sender

    # Set the recipient email address (the person who will receive the email)
    to_email = To(email = "siddharth13101999singh@gmail.com")  # Change this to your recipient

    # Define the email content with the given HTML body and the correct MIME type
    content = Content(mime_type = "text/html", content = html_body)

    # Create the email object with sender, recipient, subject, and content
    # Note: Using 'plain_text_content' to store HTML here is a mistake in SendGrid API usage.
    # The correct argument in Mail() is 'html_content' for HTML body, not 'plain_text_content'.
    mail = Mail(from_email = from_email, to_emails = to_email, subject = subject, html_content = content).get()

    # Send the email via the SendGrid client
    sg.client.mail.send.post(request_body = mail)

    # Return a success response
    return {"status" : "success"}

In [18]:
# Defining a list named 'tools' which groups together multiple tool instances
# This list includes:
# 1. subject_tool - the tool that generates an email subject from a message
# 2. html_tool - the tool that converts plain text email bodies to HTML format
# 3. send_html_email - a tool/function responsible for sending the final HTML-formatted email
# Grouping these tools together makes it easy to provide them as capabilities 
# to another agent or workflow that needs to perform multiple related actions sequentially.
tools = [subject_tool, html_tool, send_html_email]

In [19]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134cb22ade0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134f3f19f80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Sends an HTML-formatted email using the SendGrid API.\n\n<b>*Param

In [20]:
# Instructions that describe the role and workflow of the Email Manager agent:
# This agent acts as an email formatter and sender.
# It receives the raw body of an email to be sent, then performs the following steps in order:
# 1. Uses the 'subject_writer' tool to generate an effective email subject from the message content.
# 2. Uses the 'html_converter' tool to turn the plain text email body into a properly formatted HTML email body.
# 3. Uses the 'send_html_email' tool to actually send the email, combining the generated subject and HTML body.
instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


# Creating the Email Manager agent that orchestrates the email creation and sending process
# - name: Identifier for the agent
# - instructions: Defines what the agent does and how it should use its tools
# - tools: List of tools that this agent can invoke (subject writing, HTML conversion, sending email)
# - model: AI model powering this agent's reasoning and execution (GPT-4o-mini)
# - handoff_description: Describes what happens when this agent completes its task, useful for chaining workflows
emailer_agent = Agent(name = "Email Manager", instructions = instructions, tools = tools, model = "gpt-4o-mini", 
                      handoff_description = "Convert an email to HTML and send it")

### Now we have 3 tools and 1 handoff

In [ ]:
# Define a list named 'tools' which contains three sales agent tools:
# - tool1 corresponds to the Professional Sales Agent
# - tool2 corresponds to the Engaging Sales Agent
# - tool3 corresponds to the Busy Sales Agent
tools = [tool1, tool2, tool3]

# Define a list named 'handoffs' which contains agents to which control can be passed after task completion
# In this case, 'emailer_agent' is listed as the handoff agent responsible for formatting and sending emails
handoffs = [emailer_agent]

# Print the defined tools list (sales agent tools)
print(tools)

# Print the defined handoffs list (emailer agent)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134f01ced40>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7134cbb4ede0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [23]:
# Instructions defining the role and workflow of the Sales Manager agent:
# - The Sales Manager's job is to find the best cold sales email by leveraging three sales agents.
# - It must follow these steps precisely:
#   1. Use all three sales agent tools to generate three different email drafts.
#   2. Wait until all drafts are ready; do not proceed prematurely.
#   3. Evaluate all drafts and choose the single best one based on effectiveness.
#   4. The Sales Manager can repeat drafts generation if not satisfied with initial results.
#   5. It must hand off exactly one final winning email draft to the 'Email Manager' agent.
# - Important rules:
#   - The drafts must be generated by the sales agent tools; the manager does not write drafts directly.
#   - Only one email draft is handed off to the Email Manager - no multiple emails.

sales_manager_instructions = """
You are a Sales Manager at CompanionAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

# Create the Sales Manager agent instance:
# - name: "Sales Manager"
# - instructions: detailed section above explaining steps and rules
# - tools: the list of sales agent tools (tool1, tool2, tool3)
# - handoffs: the list containing the email_manager agent for handling sending
# - model: AI model used (gpt-4o-mini)
sales_manager = Agent(name = "Sales Manager", instructions = sales_manager_instructions, tools = tools, handoffs = handoffs, model = "gpt-4o-mini")


# Define the message payload to trigger sales_manager's workflow:
# This message instructs to send a cold sales email addressed to "Dear CEO" from "Alice".
message = "Send out a cold sales email addressed to Dear CEO from Alice"


# Use a tracing context labeled "Automated SDR" to track workflow execution steps
with trace("Automated SDR"):
    # Run the sales_manager agent asynchronously with the message
    # It will generate drafts, select the best one, and hand off for sending automatically
    result = await Runner.run(sales_manager, message)

<p align="center">
  <img src="./flow_chart/sales_manager_flow_diagram.png" alt="Diagram" width="700"/>
</p>


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.

In [ ]:
# @startuml AgenticAICompleteFlow

# skinparam backgroundColor #F0F8FF
# skinparam package {
#   BackgroundColor #D3E4F0
#   BorderColor #2A75BB
#   FontColor #2A75BB
#   FontSize 14
#   FontStyle bold
# }
# skinparam agent {
#   BackgroundColor #A9CCE3
#   BorderColor #1A5276
#   FontColor #154360
#   FontSize 12
#   FontStyle bold
#   Shadowing true
# }
# skinparam arrow {
#   Color #2471A3
#   Thickness 2
#   FontSize 11
#   FontStyle italic
# }

# package "CompanionAI Agentic AI System" {

#   package "Sales Agents" {
#     agent sales_agent1 as "Professional Sales Agent\n(GPT-4o-mini)"
#     agent sales_agent2 as "Engaging Sales Agent\n(GPT-4o-mini)"
#     agent sales_agent3 as "Busy Sales Agent\n(GPT-4o-mini)"
#   }

#   agent sales_manager as "Sales Manager\n(GPT-4o-mini)\n- Generate 3 drafts\n- Evaluate & select best\n- Handoff 1 email draft"

#   package "Email Management Tools" {
#     agent subject_writer as "Email Subject Writer\n(GPT-4o-mini)"
#     agent html_converter as "HTML Email Body Converter\n(GPT-4o-mini)"
#     agent send_html_email as "Send HTML Email\n(SendGrid API)"
#   }

#   agent email_manager as "Email Manager\n(GPT-4o-mini)\n- Create subject\n- Convert body to HTML\n- Send email"

#   ' Sequential interaction flow with numbered steps

#   sales_agent1 ..> sales_manager : "1. Generate draft"
#   sales_agent2 ..> sales_manager : "1. Generate draft"
#   sales_agent3 ..> sales_manager : "1. Generate draft"

#   sales_manager --> email_manager : "2. Handoff winning email draft"

#   email_manager --> subject_writer : "3a. Generate subject"
#   email_manager --> html_converter : "3b. Convert body to HTML"
#   email_manager --> send_html_email : "3c. Send HTML email"

# }

# @enduml